In [1]:
import os
import glob
import subprocess
import sys

- `subprocess` 모듈은 파이썬 프로그램 내에서 새로운 프로세스를 스폰하고 여기에 입출력 파이프를 연결하며 리턴코드를 획득할 수 있도록 하는 모듈로, 다른 언어로 만들어진 프로그램을 통합, 제어할 수 있게 만드는 모듈이다. 이 모듈은 기존에 오랜된 몇몇 모듈과 함수(`os.system`, `os.spawn*`)들을 대체하기 위해 만들어졌다. (혹은 `os.popen` 같은 함수도…)
- `sys` : 파이썬 인터프리터를 제어

This script prepare training folder and its dataset for each speaker.
- Folder `s{i}/datasets/train` would contain original DATASET_VIDEO - `s{i} with 0 <= i < VAL_SAMPLES`
- Folder `s{i}/datasets/val` would contain s{i} >= `VAL_SAMPLES`
- Folder `s{i}/datasets/align` would contain all your `*.align`

<br>

**Usage:** <br>
`$ python prepare.py [Path to video dataset] [Path to align dataset] [Number of samples(샘플 파일의 갯수)]`

> `train`폴더 안에서 `overlapped_speakers`에만 **video**폴더가 없다! `prepare.py`파일이 들어있음. <br>
> 위의 `Usage`를 사용하기 위해서는 `overlapped_speakers`를 빼고 다른 폴더의 경로를 사용하면 된다.

<br>

**Notes:**
- [Path to video dataset] should be a folder with structure: `/s{i}/[video]`
- [Path to align dataset] should be a folder with structure: `/[align].align`
- [Number of samples] should be less than or equal to `min(len(ls '/s{i}/*'))`

In [8]:
!python /home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers/prepare.py /home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers_curriculum/datasets/video /home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers_curriculum/datasets/align 5

ln: failed to create symbolic link '/home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers/s2/datasets/val/s2/s2_l_bgih4s/s2_l_bgih4s.mov': 파일이 있습니다
Traceback (most recent call last):
  File "/home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers/prepare.py", line 52, in <module>
    subprocess.check_output(path, shell=True)
  File "/usr/lib/python2.7/subprocess.py", line 223, in check_output
    raise CalledProcessError(retcode, cmd, output=output)
subprocess.CalledProcessError: Command 'ln -s '/home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers_curriculum/datasets/video/s2/s2_l_bgih4s.mov' '/home/byeongryul/다운로드/LipNet_running-main/LipNet_running/LipNet/training/overlapped_speakers/s2/datasets/val/s2/s2_l_bgih4s'' returned non-zero exit status 1


> ❗이 주피터 노트북에서 아래의 셀들을 실행하는 것이 아님❗<br>
위의 셀을 실행시켰다면, `prepare.py` 파일 전체가 실행 된 것임

In [9]:
CURRENT_PATH = os.path.dirname(os.path.abspath(__file__))

# sys.argv : 작성된 인자값을 모두 리스트로 출력함. [0]번째로는 파일명이 출력됨
# sys.argv[1], sys.argv[2], sys.argv[3] : 위의 셀에서 입력한 순서와 동일

DATASET_VIDEO_PATH = sys.argv[1]  # video path
DATASET_ALIGN_PATH = sys.argv[2]  # align path

VAL_SAMPLES = int(sys.argv[3])   # 샘플 파일 갯수

NameError: name '__file__' is not defined

![]()

In [ ]:
for speaker_path in glob.glob(os.path.join(DATASET_VIDEO_PATH, '*')):   # video 폴더에 있는 모든 폴더와 파일을 리스트로 가지고 온다.
    
    # os.path.splitext(파일 경로)[0] : 파일 경로에서 확장자(extension)만 뺀 나머지 경로
    # os.path.splitext(파일 경로)[1] : 파일 경로에서 확장자만 가지고 옴
    speaker_id = os.path.splitext(speaker_path)[0].split('/')[-1]    # 확장자만 뺀 파일명
    
    path = "mkdir -p '{}'".format(os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'train'))   
    # os.path.join() : 경로 생성. 콤마(,)로 구분된 입력들은 '/'로 구분되어 출력된다.
    # 

    if not os.path.exists(path):   # os.path.exists(파일 경로) : directory 또는 file 존재 여부 확인
        subprocess.check_output(path, shell=True)   
        # subprocess.check_output(명령, shell=True) : 명령을 쉘에서 실행하고 출력을 문자열에 저장 -> 어디에 저장하는건지..?
        # 경로가 존재하지 않는다면 명령을 실행해서 해당 디렉토리를 생성

    for s_path in glob.glob(os.path.join(DATASET_VIDEO_PATH, '*')):
        s_id = os.path.splitext(s_path)[0].split('/')[-1]

        if s_path == speaker_path:
            path = "mkdir -p '{}'".format(os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'train', s_id))   # s_path와 speaker_path가 같다면 path라는 명령을 만든다. (train)
            if not os.path.exists(path):
                subprocess.check_output(path, shell=True)  # 존재하지 않는다면 명령 실행
            
            path = "mkdir -p '{}'".format(os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'val', s_id))   # path라는 명령을 만든다.(val)
            if not os.path.exists(path):
                subprocess.check_output(path, shell=True)
            n = 0
            for video_path in glob.glob(os.path.join(DATASET_VIDEO_PATH, speaker_id, '*')):
                video_id = os.path.splitext(video_path)[0].split('/')[-1]
                if n < VAL_SAMPLES:
                    path = "ln -s '{}' '{}'".format(video_path, os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'val', s_id, video_id))
                    if not os.path.exists(path):
                        subprocess.check_output(path, shell=True)
                else:
                    path = "ln -s '{}' '{}'".format(video_path, os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'train', s_id, video_id))
                    if not os.path.exists(path):
                        subprocess.check_output(path, shell=True)
                n += 1
        else:   # s_path와 speaker_path가 같지 않을 때
            path = "ln -s '{}' '{}'".format(s_path, os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'train', s_id))
            if not os.path.exists(path):
                subprocess.check_output(path, shell=True)
    path = "ln -s '{}' '{}'".format(DATASET_ALIGN_PATH, os.path.join(CURRENT_PATH, speaker_id, 'datasets', 'align'))
    if not os.path.exists(path):
        subprocess.check_output(path, shell=True)

1. 전처리 코드를 실행하면 `training`안의 각 폴더에서 `video`폴더 안의 비디오와 같은 파일명의 폴더를 생성
2. 비디오 파일명과 같은 폴더 안에서 `.png`파일들이 생성되어 저장된다.
3. 현재 사용하고 있는 (김선아)컴퓨터 내에서는, `overlapped_speakers_curriculum`, `random_split`폴더 안의 비디오가 `.png`로 생성되어 있다.
    - `overlapped_speakers_curriculum` : 김선아 실행 (현재 이 주피터 노트북에서 실행함)
    - `random_split` : 박혜령 실행 (깃허브에 올라와져 있음)